In [1]:
import random

import carla

# Connect to the client and retrieve the world object
client = carla.Client('localhost', 2000)
# client.set_timeout(500)
world = client.get_world()

In [2]:
spectator = world.get_spectator()

# Get the location and rotation of the spectator through its transform
transform = spectator.get_transform()


# Set the spectator with an empty transform
#spectator.set_transform(carla.Transform())
spawn_points = world.get_map().get_spawn_points()


In [3]:
spectator.set_transform(spawn_points[0])

In [ ]:
#python config.py --map Town02
#cd dl/cap/carla/windowsnoeditor/pythonapi/util

In [2]:
spawn_points = world.get_map().get_spawn_points()
spawn_0 = spawn_points[0]
spawn_points = spawn_points[1:]
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')
my_vehicle = world.spawn_actor(random.choice(vehicle_blueprints), spawn_0)

In [ ]:
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints, random.choice(spawn_points)))
for vehicle in world.get_actors().filter('vehicle'):
    vehicle.set_autopilot(True)

In [3]:
blueprint = world.get_blueprint_library().find('sensor.camera.rgb')
# Modify the attributes of the blueprint to set image resolution and field of view.
blueprint.set_attribute('image_size_x', '1920')
blueprint.set_attribute('image_size_y', '1080')
blueprint.set_attribute('fov', '110')
#blueprint.set_attribute('sensor_tick', '1.5')
blueprint.set_attribute('blur_amount', '0')
#motion_blur_max_distortion
blueprint.set_attribute('motion_blur_max_distortion', '0')
# Set the time in seconds between sensor captures
blueprint.set_attribute('sensor_tick', '3.0')
blueprint.set_attribute('shutter_speed', '100.0')
transform = carla.Transform(carla.Location(x=0.8, z=1.7))
sensor = world.spawn_actor(blueprint, transform, attach_to=my_vehicle)
#sensor.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))


In [6]:
spectator = world.get_spectator()
spectator.set_transform(spawn_0)

In [4]:
my_vehicle.set_autopilot(True)

## Cars

In [1]:
import carla
import random
import queue
import numpy as np
import cv2

client = carla.Client('localhost', 2000)
world  = client.get_world()
bp_lib = world.get_blueprint_library()

In [2]:
def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

def get_image_point(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # New we must change from UE4's coordinate system to an "standard"
        # (x, y ,z) -> (y, -z, x)
        # and we remove the fourth componebonent also
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)
        # normalize
        point_img[0] /= point_img[2]
        point_img[1] /= point_img[2]

        return point_img[0:2]

In [3]:
# spawn vehicle
spawn_points = world.get_map().get_spawn_points()
vehicle_bp =bp_lib.find('vehicle.lincoln.mkz_2020')
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

for i in range(50):
    vehicle_bp = random.choice(bp_lib.filter('vehicle'))
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
    if npc:
        npc.set_autopilot(True)

# spawn camera
camera_bp = bp_lib.find('sensor.camera.rgb')
# camera_bp.set_attribute('image_size_x', '1920')
# camera_bp.set_attribute('image_size_y', '1080')
# camera_bp.set_attribute('fov', '110')
camera_init_trans = carla.Transform(carla.Location(z=2))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
vehicle.set_autopilot(True)

# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 5
world.apply_settings(settings)

# Create a queue to store and retrieve the sensor data
image_queue = queue.Queue()
camera.listen(image_queue.put)

In [4]:
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

# Get the attributes from the camera
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
fov = camera_bp.get_attribute("fov").as_float()

# Calculate the camera projection matrix to project from 3D -> 2D
K = build_projection_matrix(image_w, image_h, fov)

In [5]:
world.tick()
image = image_queue.get()
img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
#actor_list = world.get_actors()
# Get camera matrix
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
labs = []
x_mins=[]
y_mins=[]
x_maxs=[]
y_maxs=[]
files = []

#actor_list = world.get_actors()
actor_list = []
for i in world.get_actors().filter('*vehicle*'):
    actor_list.append(i)
for i in world.get_actors().filter('*traffic*'):
    actor_list.append(i)
print(actor_list)

for curr in actor_list:
    if curr.id != vehicle.id:
        bb = curr.bounding_box
        dist = curr.get_transform().location.distance(vehicle.get_transform().location)
        print(dist)
        if dist < 50:
            print("test")
            forward_vec = vehicle.get_transform().get_forward_vector()
            ray = curr.get_transform().location - vehicle.get_transform().location

            if forward_vec.dot(ray) > 1:
                lab = ""
                lab = curr.type_id
                p1 = get_image_point(bb.location, K,world_2_camera)
                verts = [v for v in bb.get_world_vertices(curr.get_transform())]
                x_max = -10000
                x_min = 10000
                y_max = -10000
                y_min = 10000
                for vert in verts:
                    p = get_image_point(vert, K, world_2_camera)
                    print(p)
                    # Find the rightmost vertex
                    if p[0] > x_max:
                        x_max = p[0]
                    # Find the leftmost vertex
                    if p[0] < x_min:
                        x_min = p[0]
                    # Find the highest vertex
                    if p[1] > y_max:
                        y_max = p[1]
                    # Find the lowest  vertex
                    if p[1] < y_min:
                        y_min = p[1]
                labs.append(lab)
                x_mins.append(x_min)
                x_maxs.append(x_max)
                y_mins.append(y_min)
                y_maxs.append(y_max)




                cv2.line(img, (int(x_min),int(y_min)), (int(x_max),int(y_min)), (0,0,255, 255), 1)
                cv2.line(img, (int(x_min),int(y_max)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
                cv2.line(img, (int(x_min),int(y_min)), (int(x_min),int(y_max)), (0,0,255, 255), 1)
                cv2.line(img, (int(x_max),int(y_min)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
cv2.imwrite("out/test.png", img)


[<carla.libcarla.Vehicle object at 0x000001A47121C7B0>, <carla.libcarla.Vehicle object at 0x000001A47121CDB0>, <carla.libcarla.Vehicle object at 0x000001A47121CE70>, <carla.libcarla.Vehicle object at 0x000001A47121CED0>, <carla.libcarla.Vehicle object at 0x000001A47121CB70>, <carla.libcarla.Vehicle object at 0x000001A47121CC30>, <carla.libcarla.Vehicle object at 0x000001A47121CBD0>, <carla.libcarla.Vehicle object at 0x000001A47121C390>, <carla.libcarla.Vehicle object at 0x000001A47121CAB0>, <carla.libcarla.Vehicle object at 0x000001A47121C990>, <carla.libcarla.Vehicle object at 0x000001A47121C930>, <carla.libcarla.Vehicle object at 0x000001A47121C5D0>, <carla.libcarla.Vehicle object at 0x000001A47121C270>, <carla.libcarla.Vehicle object at 0x000001A47121C690>, <carla.libcarla.Vehicle object at 0x000001A47121C8D0>, <carla.libcarla.Vehicle object at 0x000001A47121C750>, <carla.libcarla.Vehicle object at 0x000001A47121C510>, <carla.libcarla.Vehicle object at 0x000001A47121C6F0>, <carla.li

True

In [5]:
world.tick()
image = image_queue.get()

img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# # Get the camera matrix
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

for npc in world.get_actors().filter('*vehicle*'):

    # Filter out the ego vehicle
    if npc.id != vehicle.id:

        bb = npc.bounding_box
        dist = npc.get_transform().location.distance(vehicle.get_transform().location)

        # Filter for the vehicles within 50m
        if dist < 50:

            # Calculate the dot product between the forward vector
            # of the vehicle and the vector between the vehicle
            # and the other vehicle. We threshold this dot product
            # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
            forward_vec = vehicle.get_transform().get_forward_vector()
            ray = npc.get_transform().location - vehicle.get_transform().location

            if forward_vec.dot(ray) > 1:
                p1 = get_image_point(bb.location, K, world_2_camera)
                verts = [v for v in bb.get_world_vertices(npc.get_transform())]
                x_max = -10000
                x_min = 10000
                y_max = -10000
                y_min = 10000

                for vert in verts:
                    p = get_image_point(vert, K, world_2_camera)
                    # Find the rightmost vertex
                    if p[0] > x_max:
                        x_max = p[0]
                    # Find the leftmost vertex
                    if p[0] < x_min:
                        x_min = p[0]
                    # Find the highest vertex
                    if p[1] > y_max:
                        y_max = p[1]
                    # Find the lowest  vertex
                    if p[1] < y_min:
                        y_min = p[1]

                cv2.line(img, (int(x_min),int(y_min)), (int(x_max),int(y_min)), (0,0,255, 255), 1)
                cv2.line(img, (int(x_min),int(y_max)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
                cv2.line(img, (int(x_min),int(y_min)), (int(x_min),int(y_max)), (0,0,255, 255), 1)
                cv2.line(img, (int(x_max),int(y_min)), (int(x_max),int(y_max)), (0,0,255, 255), 1)

cv2.imwrite("out/test.png",img)

True

## Testing 3D

In [5]:
world.tick()
edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]]
image = image_queue.get()

img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# Get the camera matrix
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())


for npc in world.get_actors().filter('*vehicle*'):

    # Filter out the ego vehicle
    if npc.id != vehicle.id:

        bb = npc.bounding_box
        dist = npc.get_transform().location.distance(vehicle.get_transform().location)

        # Filter for the vehicles within 50m
        if dist < 50:

            # Calculate the dot product between the forward vector
            # of the vehicle and the vector between the vehicle
            # and the other vehicle. We threshold this dot product
            # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
            forward_vec = vehicle.get_transform().get_forward_vector()
            ray = npc.get_transform().location - vehicle.get_transform().location

            if forward_vec.dot(ray) > 1:
                p1 = get_image_point(bb.location, K, world_2_camera)
                verts = [v for v in bb.get_world_vertices(npc.get_transform())]
                for edge in edges:
                    p1 = get_image_point(verts[edge[0]], K, world_2_camera)
                    p2 = get_image_point(verts[edge[1]],  K, world_2_camera)
                    cv2.line(img, (int(p1[0]),int(p1[1])), (int(p2[0]),int(p2[1])), (255,0,0, 255), 1)
cv2.imwrite("out/test.png",img)

True

## BB Stuff
Realistically we can only get 3 classes TrafficSigns, Pedestrians, and Vehicles

In [4]:
import queue
world  = client.get_world()
bp_lib = world.get_blueprint_library()

# spawn vehicle
vehicle_bp =bp_lib.find('vehicle.lincoln.mkz_2020')
spawn_points = world.get_map().get_spawn_points()
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])

# spawn camera
camera_bp = bp_lib.find('sensor.camera.rgb')
#camera_bp.set_attribute('sensor_tick', '3.0')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '110')
camera_init_trans = carla.Transform(carla.Location(z=2))
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)
vehicle.set_autopilot(True)

# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True # Enables synchronous mode
settings.fixed_delta_seconds = 5
world.apply_settings(settings)

# Get the map spawn points
spawn_points = world.get_map().get_spawn_points()

# Create a queue to store and retrieve the sensor data
image_queue = queue.Queue()
camera.listen(image_queue.put)

In [6]:
import numpy as np
def build_projection_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

def get_image_point(loc, K, w2c):
        # Calculate 2D projection of 3D coordinate

        # Format the input coordinate (loc is a carla.Position object)
        point = np.array([loc.x, loc.y, loc.z, 1])
        # transform to camera coordinates
        point_camera = np.dot(w2c, point)

        # New we must change from UE4's coordinate system to an "standard"
        # (x, y ,z) -> (y, -z, x)
        # and we remove the fourth componebonent also
        point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

        # now project 3D->2D using the camera matrix
        point_img = np.dot(K, point_camera)
        # normalize
        point_img[0] /= point_img[2]
        point_img[1] /= point_img[2]

        return point_img[0:2]

In [7]:
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())

# Get the attributes from the camera
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
fov = camera_bp.get_attribute("fov").as_float()

# Calculate the camera projection matrix to project from 3D -> 2D
K = build_projection_matrix(image_w, image_h, fov)

In [8]:
bounding_box_set = world.get_level_bbs(carla.CityObjectLabel.TrafficLight)
bounding_box_set.extend(world.get_level_bbs(carla.CityObjectLabel.TrafficSigns))

# Remember the edge pairs
edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]]

In [8]:
actor_list = world.get_actors()

for i in world.get_actors():
    print(i.get_location(), i.type_id)

Location(x=108.279999, y=307.859985, z=4.310000) spectator
Location(x=147.110016, y=234.222107, z=0.220000) traffic.traffic_light
Location(x=129.550003, y=226.559998, z=0.220000) traffic.traffic_light
Location(x=122.480003, y=243.816986, z=0.220000) traffic.traffic_light
Location(x=121.369987, y=194.089951, z=0.220000) traffic.traffic_light
Location(x=138.929977, y=201.819962, z=0.220000) traffic.traffic_light
Location(x=146.000000, y=184.829971, z=0.220000) traffic.traffic_light
Location(x=30.959990, y=194.289963, z=0.220000) traffic.traffic_light
Location(x=48.519978, y=203.279953, z=0.220000) traffic.traffic_light
Location(x=58.050007, y=185.029984, z=0.220000) traffic.traffic_light
Location(x=186.950058, y=176.769974, z=0.220000) traffic.traffic_light
Location(x=179.220016, y=194.329910, z=0.220000) traffic.traffic_light
Location(x=196.210022, y=201.400040, z=0.220000) traffic.traffic_light
Location(x=186.950058, y=226.179977, z=0.220000) traffic.traffic_light
Location(x=179.220016

In [9]:
actor_list = world.get_actors()
actors = []
for i in world.get_actors().filter('*vehicle*'):
    actors.append(i)
for i in world.get_actors().filter('*traffic*'):
    actors.append(i)
actors

In [12]:
location = world.get_level_bbs(carla.CityObjectLabel.TrafficSigns)[0].location
location.x, location.y

(108.19998168945312, 333.25)

## Some issues with conversion from 3D bb to 2

In [6]:
import cv2
#world.tick()
bounding_box_set = world.get_level_bbs(carla.CityObjectLabel.TrafficLight)
bounding_box_set.extend(world.get_level_bbs(carla.CityObjectLabel.TrafficSigns))
edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]]
image = image_queue.get()
img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
actor_list = world.get_actors()
# Get camera matrix
world_2_camera = np.array(camera.get_transform().get_inverse_matrix())
labs = []
x_mins=[]
y_mins=[]
x_maxs=[]
y_maxs=[]
files = []
for bb in bounding_box_set:
    dist = bb.location.distance(vehicle.get_transform().location)
    if dist < 30:
        forward_vec = vehicle.get_transform().get_forward_vector()
        ray = bb.location - vehicle.get_transform().location

        if forward_vec.dot(ray) > 1:
            lab = ""
            location_x, location_y = bb.location.x, bb.location.y
            print(location_x, location_y)
            for i in actor_list:
                tmp_x, tmp_y = i.get_location().x, i.get_location().y
                if round(location_x) == round(tmp_x) and round(location_y) == round(tmp_y):
                    lab = i.type_id
                    print(lab)
            p1 = get_image_point(bb.location, K,world_2_camera)
            verts = [v for v in bb.get_world_vertices(carla.Transform())]
            x_max = -10000
            x_min = 10000
            y_max = -10000
            y_min = 10000
            for vert in verts:
                p = get_image_point(vert, K, world_2_camera)
                print(p)
                # Find the rightmost vertex
                if p[0] > x_max:
                    x_max = p[0]
                # Find the leftmost vertex
                if p[0] < x_min:
                    x_min = p[0]
                # Find the highest vertex
                if p[1] > y_max:
                    y_max = p[1]
                # Find the lowest  vertex
                if p[1] < y_min:
                    y_min = p[1]
            labs.append(lab)
            x_mins.append(x_min)
            x_maxs.append(x_max)
            y_mins.append(y_min)
            y_maxs.append(y_max)



            cv2.line(img, (int(x_min),int(y_min)), (int(x_max),int(y_min)), (0,0,255, 255), 1)
            cv2.line(img, (int(x_min),int(y_max)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
            cv2.line(img, (int(x_min),int(y_min)), (int(x_min),int(y_max)), (0,0,255, 255), 1)
            cv2.line(img, (int(x_max),int(y_min)), (int(x_max),int(y_max)), (0,0,255, 255), 1)
cv2.imwrite("out/test.png", img)

187.15576171875 225.91940307617188
traffic.traffic_light
[292.69756959 303.99074781]
[292.62760959 285.2005885 ]
[294.42336688 303.95584007]
[294.35421797 285.48585001]
[288.41489252 304.0124338 ]
[288.34588316 285.21925174]
[290.21396159 303.9771509 ]
[290.14573107 285.50424023]
178.96542358398438 243.53421020507812
traffic.traffic_light
[-285.60635823  309.96850311]
[-285.42671956  253.61755497]
[-303.60764783  310.06167371]
[-303.41576029  253.6734095 ]
[-315.06092341  310.31637571]
[-314.85281026  251.52447575]
[-333.86331331  310.41382751]
[-333.64183754  251.58130746]
196.0043182373047 251.07061767578125
traffic.traffic_light
[952.02694802 327.22113422]
[948.07151736 -13.77459392]
[1209.58713184  338.88782405]
[1202.28628803 -158.61939925]
[1027.85600763  326.75936733]
[1023.60407406  -13.24030398]
[1319.16306003  338.16400313]
[1311.24475728 -157.22581094]


True

In [5]:
from time import sleep
sensor.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))
sleep(10)
sensor.stop()

In [5]:
sensor.stop()

In [5]:
# Create a transform to place the camera on top of the vehicle
camera_init_trans = carla.Transform(carla.Location(z=1.5))

# We create the camera through a blueprint that defines its properties
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')

# We spawn the camera and attach it to our ego vehicle
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

In [6]:
camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))